In [1]:
import pandas as pd
import math
from pyproj import Proj, transform


bb = pd.read_csv("bb.csv")
population = [pd.read_csv("population_0.csv"), pd.read_csv("population_1.csv")]

sweref = Proj(init='epsg:3006')
wgs84 = Proj(init='epsg:4326')

for i, pop in enumerate(population):
    coords = pop.apply(lambda x: pd.Series(transform(sweref, wgs84, x.X, x.Y)), axis=1)
    coords.columns = ["x", "y"]
    population[i] = pop.join(coords)

def getDistance(k1, k2):
    lat1 = float(k1["lat"])
    lon1 = float(k1["lon"])
    lat2 = float(k2["lat"])
    lon2 = float(k2["lon"])

    radius = 6371  # km

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    d = radius * c

    return math.floor(d)



In [2]:
curr_date = "2017-05-26"
bb = bb[(bb.edate.astype(str) > curr_date) & (bb.sdate.astype(str) <= curr_date)]

def nearest_bb(x, y):
    mindist = bb.apply(lambda row: getDistance({"lat": row.Y, "lon": row.X}, {"lat": y, "lon": x}), axis=1).min()
    return mindist
population[1]["dist_to_nearest_bb"] = population[1].apply(lambda ruta: nearest_bb(ruta.x, ruta.y), axis=1)
population[1]


,Ruta,Pop,KNKOD,KNNAMN,X,Y,x,y,dist_to_nearest_bb
0,3770006453000,5,1470,Vara,377500,6453500,12.915397,58.205645,40
1,5970006807000,84,2182,Sderhamn,597500,6807500,16.825228,61.389173,40
2,5800006810000,1,2183,Bollns,580500,6810500,16.508425,61.419992,46
3,5550006482000,10,581,Norrkping,555500,6482500,15.951718,58.479596,16
4,5620006977000,2,2305,Brcke,562500,6977500,16.230671,62.922032,79
5,4320006357000,11,617,Gnosj,432500,6357500,13.878105,57.355355,22
6,4140006325000,2,1315,Hylte,414500,6325500,13.590028,57.064960,29
7,3890006221000,7,1276,Klippan,389500,6221500,13.222368,56.125761,33
8,3240006534000,34,1438,Dals-Ed,324500,6534500,11.952197,58.914226,68
9,3840006477000,16,1494,Lidkping,384500,6477500,13.022442,58.422964,45


In [3]:
# Genomsnitt i riket
total_pop = population[1].Pop.sum()
(population[1].dist_to_nearest_bb * (population[1].Pop / total_pop)).sum()

17.37712097896758

In [4]:
# Antal över 100 km
population[1][population[1].dist_to_nearest_bb > 100].Pop.sum()

72665

In [5]:
# Genomsnitt i Arjeplog
arjeplog = population[1][population[1].KNNAMN == "Arjeplog"]
total_pop = arjeplog.Pop.sum()
(arjeplog.dist_to_nearest_bb * (arjeplog.Pop / total_pop)).sum()

164.16732072779166

In [6]:
# Genomsnitt i Solna
solna = population[1][population[1].KNNAMN == "Solna"]
total_pop = solna.Pop.sum()
(solna.dist_to_nearest_bb * (solna.Pop / total_pop)).sum()

2.0345329071027756

In [7]:
# Max i Alingsås
population[1][population[1].KNNAMN == "Alingss"].dist_to_nearest_bb.max()

39